# TF IDF

## Requirements 

In [4]:
# Imports
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline


import numpy as np
import pandas as pd

import _helpers as hp

## Load Data

In [5]:
ASR_data = pd.read_csv("snips/new_ASR_with_labels.csv") # ASR data with improved speech recognition with 15555 framerate and with autocorrection applied

GT_data = pd.read_csv("snips/merged_GT_data.csv") # Groundtruth data

### DATA manipulation

In [6]:
#Get ASR data into a numpy word array per sentence plus a numpy label array

XX_ASR = ASR_data["transcript"].apply(lambda x: x.split(' '))
X_ASR = list(XX_ASR) #numpy word array per transcript

y_num_ASR = np.array(ASR_data["user_action_num"]) #labels


In [7]:
#Get Groundtruth data into a numpy word array per sentence plus a numpy label array

XX_GT = GT_data["transcript"].apply(lambda x: x.split(' '))
X_GT = list(XX_GT)  #numpy word array per transcript

y_num_GT = np.array(GT_data["user_action_num"]) #labels

In [8]:
# concat the 2 arrays into one
X_all = X_ASR + X_GT

## Keyword to user action

Label each sentence in the dataset acording with keywords to user action

## TF IDF vector representation 

### Create TD IDF model based in the dictionary of our dataset 

In [9]:
# Create TD*IDF vector represenation
dct_X = Dictionary(X_all)
corpusBOW = [dct_X.doc2bow(line) for line in X_all]
model = TfidfModel(corpusBOW)


corpusBOW_ASR = [dct_X.doc2bow(line) for line in X_ASR]
X_ASR_vec = model[corpusBOW_ASR]


corpusBOW_GT = [dct_X.doc2bow(line) for line in X_GT]
X_GT_vec = model[corpusBOW_GT]

In [10]:
def Tfidf2np(X,dct):
    """ 
    Converts gensim format to numpy array
    Input:
    X - TDidfModel vector (N x lenght("sentence"))
    dct - Dictionary object (lenght("unique words"))
    Output:
    X_np - N x length("unique words")
    """
    N_dict = len(dct)
    N_sent = len(X)
    X_np = np.zeros((N_sent,N_dict))
    i = 0
    for _list in X:
        for word in _list:
            X_np[i, word[0]] = word[1]
        i += 1
    return X_np

In [11]:
#Tokenize data
X_ASR_np = Tfidf2np(X_ASR_vec,dct_X)
X_GT_np = Tfidf2np(X_GT_vec,dct_X)

Lets create a function that vectorizes a sentence based in a embeded text model

In [12]:
def get_TFIDF_feature(sent,dct_X, model):
    """return a vector representative of a string"""

    corpusBOW_sent = [dct_X.doc2bow(line) for line in [sent]]

    sent_vec = model[corpusBOW_sent]

    sent_np = Tfidf2np(sent_vec,dct_X)

    return sent_np

# Classifiers

In [13]:
# Split into training and test data
train_features, test_features, train_labels, test_labels = train_test_split(X_ASR_np, y_num_ASR, train_size= 0.9)

For train we will use both the training set of ASR data plus all the groudtruth data in order to have more data and build more robust models

In [14]:
# Concatenate the ASR training set plus the GroundTruth set in order to "augment" our training set
train_features = np.concatenate((train_features,X_GT_np), axis=0)

train_labels = np.concatenate((train_labels,y_num_GT), axis=0)

## Logistic Regression

In [15]:
lgr = LogisticRegression(max_iter= 1000) #Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr) #Scale feature space
lgr_pipe.fit(train_features, train_labels)


lgr_pred_labels = lgr_pipe.predict(test_features) #predictions

lgr_score = lgr_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [16]:
print(classification_report(test_labels, lgr_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.76      0.73      0.75        26
     SwitchLightOn       0.64      0.95      0.77        19
IncreaseBrightness       0.89      0.76      0.82        33
DecreaseBrightness       0.81      0.68      0.74        19
SetLightBrightness       0.86      0.95      0.90        40
     SetLightColor       0.88      0.76      0.81        29

          accuracy                           0.81       166
         macro avg       0.81      0.80      0.80       166
      weighted avg       0.82      0.81      0.81       166

[[19  4  1  0  2  0]
 [ 0 18  0  0  0  1]
 [ 1  3 25  2  1  1]
 [ 2  2  1 13  0  1]
 [ 1  0  1  0 38  0]
 [ 2  1  0  1  3 22]]

ACCURACY: 0.8132530120481928


## Naive Bayes

### Multinomial Naïve Bayes

In [17]:
mnb = MultinomialNB() #Create NB model

mnb_pipe = make_pipeline(preprocessing.Normalizer(), mnb) #Scale feature space
mnb_pipe.fit(train_features, train_labels)


mnb_pred_labels = mnb_pipe.predict(test_features) #predictions

mnb_score = mnb_pipe.score(test_features,test_labels) #accuracy

#### Model Evaluation

In [18]:
print(classification_report(test_labels, mnb_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, mnb_pred_labels))

print("\nACCURACY:", mnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.77      0.88      0.82        26
     SwitchLightOn       0.72      0.95      0.82        19
IncreaseBrightness       0.88      0.85      0.86        33
DecreaseBrightness       0.81      0.68      0.74        19
SetLightBrightness       0.95      0.88      0.91        40
     SetLightColor       0.96      0.86      0.91        29

          accuracy                           0.86       166
         macro avg       0.85      0.85      0.84       166
      weighted avg       0.87      0.86      0.86       166

[[23  0  1  0  2  0]
 [ 0 18  1  0  0  0]
 [ 1  3 28  0  0  1]
 [ 2  3  1 13  0  0]
 [ 3  0  1  1 35  0]
 [ 1  1  0  2  0 25]]

ACCURACY: 0.8554216867469879


### Bernoulli naïve Bayes

In [19]:
bnb = BernoulliNB() #Create the classification model

bnb_pipe = make_pipeline(preprocessing.Normalizer(), bnb) #Scale feature space
bnb_pipe.fit(train_features, train_labels)


bnb_pred_labels = bnb_pipe.predict(test_features) #predictions

bnb_score = bnb_pipe.score(test_features,test_labels) #accuracy

#### Model Evaluation

In [20]:
print(classification_report(test_labels, bnb_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, bnb_pred_labels))

print("\nACCURACY:", bnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.82      0.69      0.75        26
     SwitchLightOn       0.85      0.89      0.87        19
IncreaseBrightness       0.83      0.88      0.85        33
DecreaseBrightness       0.71      0.79      0.75        19
SetLightBrightness       0.95      0.90      0.92        40
     SetLightColor       0.90      0.93      0.92        29

          accuracy                           0.86       166
         macro avg       0.84      0.85      0.84       166
      weighted avg       0.86      0.86      0.85       166

[[18  1  1  3  2  1]
 [ 1 17  1  0  0  0]
 [ 1  1 29  1  0  1]
 [ 1  1  1 15  0  1]
 [ 1  0  2  1 36  0]
 [ 0  0  1  1  0 27]]

ACCURACY: 0.8554216867469879


### Gaussian Naive Bayes

In [21]:
gnb = GaussianNB() #Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb) #Scale feature space
gnb_pipe.fit(train_features, train_labels)


gnb_pred_labels = gnb_pipe.predict(test_features) #predictions

gnb_score = gnb_pipe.score(test_features,test_labels) #accuracy

#### Model Evaluation

In [22]:
print(classification_report(test_labels, gnb_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.80      0.62      0.70        26
     SwitchLightOn       0.37      0.89      0.52        19
IncreaseBrightness       0.67      0.30      0.42        33
DecreaseBrightness       0.61      0.58      0.59        19
SetLightBrightness       0.82      0.82      0.82        40
     SetLightColor       0.78      0.72      0.75        29

          accuracy                           0.65       166
         macro avg       0.68      0.66      0.63       166
      weighted avg       0.70      0.65      0.65       166

[[16  3  1  0  4  2]
 [ 0 17  0  0  0  2]
 [ 0 17 10  3  2  1]
 [ 1  5  1 11  0  1]
 [ 2  2  2  1 33  0]
 [ 1  2  1  3  1 21]]

ACCURACY: 0.6506024096385542


# SVM

In [23]:
svm = SVC() #Create the classification model

svm_pipe = make_pipeline(preprocessing.Normalizer(), svm) #Scale feature space
svm_pipe.fit(train_features, train_labels)


svm_pred_labels = svm_pipe.predict(test_features) #predictions

svm_score = svm_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [24]:
print(classification_report(test_labels, svm_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.85      0.88      0.87        26
     SwitchLightOn       0.86      1.00      0.93        19
IncreaseBrightness       0.89      0.94      0.91        33
DecreaseBrightness       0.94      0.84      0.89        19
SetLightBrightness       0.97      0.93      0.95        40
     SetLightColor       0.96      0.90      0.93        29

          accuracy                           0.92       166
         macro avg       0.91      0.91      0.91       166
      weighted avg       0.92      0.92      0.92       166

[[23  1  1  0  1  0]
 [ 0 19  0  0  0  0]
 [ 1  0 31  0  0  1]
 [ 1  1  1 16  0  0]
 [ 1  0  1  1 37  0]
 [ 1  1  1  0  0 26]]

ACCURACY: 0.9156626506024096


# NEURAL NETWORK

In [25]:
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100), activation='relu', solver='adam', max_iter=5000) #Create the classification model

mlp_pipe = make_pipeline(preprocessing.Normalizer(), mlp) #Scale feature space
mlp_pipe.fit(train_features, train_labels)


mlp_pred_labels = mlp_pipe.predict(test_features) #predictions

mlp_score = mlp_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [26]:
print(classification_report(test_labels, mlp_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.77      0.77      0.77        26
     SwitchLightOn       0.70      1.00      0.83        19
IncreaseBrightness       0.88      0.85      0.86        33
DecreaseBrightness       0.94      0.79      0.86        19
SetLightBrightness       0.95      0.90      0.92        40
     SetLightColor       0.89      0.83      0.86        29

          accuracy                           0.86       166
         macro avg       0.85      0.86      0.85       166
      weighted avg       0.87      0.86      0.86       166

[[20  4  1  0  1  0]
 [ 0 19  0  0  0  0]
 [ 1  2 28  1  0  1]
 [ 1  1  1 15  0  1]
 [ 2  0  1  0 36  1]
 [ 2  1  1  0  1 24]]

ACCURACY: 0.8554216867469879


# Try Your Self

In [27]:
def user_friendly(sentence, cls):
    """return action from sentence"""
    
    sent = sentence.split()
    new_sent = []
    for word in sent:
        new_sent.append(hp.autocorrection(word))

    sent_np = get_TFIDF_feature(new_sent,dct_X,model)
    y_pred = cls.predict(sent_np)
    return hp.indx2action(y_pred)

In [28]:
user_friendly("I want a increase in the lights of my living room", lgr_pipe)

['IncreaseBrightness']